This notebook generates a displacement adjacement matrix for stations.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from functools import reduce
from geopy import Point
from geopy import distance

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
class AnalysisConfig:
    def __init__(self, csv_output_path='', stations_path=''):
        self.csv_output_path = csv_output_path
        self.stations_path = stations_path

In [3]:
STATIONS_DIR = '../data/exports'
SHAPEFILE_DIR = '../data/shapefiles/zipcodes'

config_sf = AnalysisConfig(
    csv_output_path='{}/displacement_adjacency_matrix_san_francisco.csv'.format(
        STATIONS_DIR),
    stations_path='{}/stations_san_francisco.csv'.format(STATIONS_DIR),
)

config_dc = AnalysisConfig(
    csv_output_path='{}/displacement_adjacency_matrix_washington_dc.csv'.format(
        STATIONS_DIR),
    stations_path='{}/stations_washington_dc.csv'.format(STATIONS_DIR),
)

In [4]:
config = config_sf

In [5]:
def get_distance(point1, point2):
    return distance.geodesic(point1, point2).m


def get_point(station_id):
    return Point(stations_df.loc[station_id]['lat'],
                 stations_df.loc[station_id]['lng'])


def get_displacement(station_id1, station_id2):
    return get_distance(get_point(station_id1), get_point(station_id2))

In [6]:
stations_df = pd.read_csv(config.stations_path)
stations_df.set_index('station_id', inplace=True)
stations_df.head()

,lat,lng
station_id,,
SF I29-1,37.783375,-122.393276
SF-A19,37.805339,-122.437250
SF-A20,37.804438,-122.433527
SF-A27,37.804778,-122.403261
SF-AA15,37.720570,-122.447685


In [7]:
adj_matrix = pd.DataFrame(index=stations_df.index, columns=stations_df.index)
adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
    lambda col_name: get_displacement(row.name, col_name)),
                              axis=1)
adj_matrix.head()

station_id,SF I29-1,SF-A19,SF-A20,SF-A27,SF-AA15,SF-AA28,SF-B19,SF-B21,SF-B22,SF-B25,...,SF-Y14,SF-Y25,SF-Y27,SF-Y29,SF-Y30,SF-Z16,SF-Z28,SF-Z6,SF-Z7,SF Depot
station_id,,,,,,,,,,,,,,,,,,,,,
SF I29-1,0.000000,4576.455493,4246.657658,2533.079126,8460.818672,6742.713699,4333.192056,3425.808495,3455.044732,2788.856544,...,8232.973784,6352.180868,5933.370907,6006.016706,6021.828671,8100.428181,6435.095582,10419.142740,10237.101539,3182.307168
SF-A19,4576.455493,0.000000,342.755147,2993.885998,9453.550236,9991.500782,316.374899,1152.622763,1179.705764,2120.070122,...,8909.361607,9088.214264,9012.103593,9299.604095,9540.626116,9223.905533,9656.082470,10079.767290,10056.394190,6994.544327
SF-A20,4246.657658,342.755147,0.000000,2665.682943,9391.935738,9773.646811,298.473799,821.000932,836.963820,1782.643505,...,8866.325864,8899.956912,8800.166513,9074.967390,9301.872618,9150.632386,9438.397059,10124.233338,10088.424723,6721.292004
SF-A27,2533.079126,2993.885998,2665.682943,0.000000,10133.051124,9167.128187,2898.022783,2050.160663,1882.426257,919.530556,...,9761.143574,8622.378546,8298.173862,8429.889203,8496.613177,9815.974948,8847.928878,11551.607105,11439.415964,5668.955964
SF-AA15,8460.818672,9453.550236,9391.935738,10133.051124,0.000000,4871.571998,9167.674481,9052.260326,9301.733631,9611.817292,...,733.585449,3689.254526,4437.408495,4914.675516,5477.273202,399.574322,4762.601585,3095.986459,2742.299362,6367.518843


In [8]:
adj_matrix.to_csv(config.csv_output_path)

In [9]:
# Example for retrieving displacement from SF.
csv_adj_matrix = pd.read_csv(config.csv_output_path).set_index('station_id')
display(csv_adj_matrix.head())

csv_adj_matrix['SF-AA15']['SF-Y14']

,SF I29-1,SF-A19,SF-A20,SF-A27,SF-AA15,SF-AA28,SF-B19,SF-B21,SF-B22,SF-B25,...,SF-Y14,SF-Y25,SF-Y27,SF-Y29,SF-Y30,SF-Z16,SF-Z28,SF-Z6,SF-Z7,SF Depot
station_id,,,,,,,,,,,,,,,,,,,,,
SF I29-1,0.000000,4576.455493,4246.657658,2533.079126,8460.818672,6742.713699,4333.192056,3425.808495,3455.044732,2788.856544,...,8232.973784,6352.180868,5933.370907,6006.016706,6021.828671,8100.428181,6435.095582,10419.142740,10237.101539,3182.307168
SF-A19,4576.455493,0.000000,342.755147,2993.885998,9453.550236,9991.500782,316.374899,1152.622763,1179.705764,2120.070122,...,8909.361607,9088.214264,9012.103593,9299.604095,9540.626116,9223.905533,9656.082470,10079.767290,10056.394190,6994.544327
SF-A20,4246.657658,342.755147,0.000000,2665.682943,9391.935738,9773.646811,298.473799,821.000932,836.963820,1782.643505,...,8866.325864,8899.956912,8800.166513,9074.967390,9301.872618,9150.632386,9438.397059,10124.233338,10088.424723,6721.292004
SF-A27,2533.079126,2993.885998,2665.682943,0.000000,10133.051124,9167.128187,2898.022783,2050.160663,1882.426257,919.530556,...,9761.143574,8622.378546,8298.173862,8429.889203,8496.613177,9815.974948,8847.928878,11551.607105,11439.415964,5668.955964
SF-AA15,8460.818672,9453.550236,9391.935738,10133.051124,0.000000,4871.571998,9167.674481,9052.260326,9301.733631,9611.817292,...,733.585449,3689.254526,4437.408495,4914.675516,5477.273202,399.574322,4762.601585,3095.986459,2742.299362,6367.518843


733.5854485466258